In [23]:
#Load datafile and import packages

fdir = "/home/julian/repositories/RTK_on_the_beach/hanglog/soper"

import os, sys, datetime
sys.path.append("..")
import hacktrack.loaders
import pandas, numpy, math
from scipy import optimize
from hacktrack import loaders, utils

logfiles = os.listdir(fdir)
print(logfiles)
fLog = os.path.join(fdir, logfiles[0])
fd = loaders.FlyDat(fLog)

rectypespresent = [k for k in fd.reccounts  if fd.reccounts[k] != 0]
fd.LoadC("".join(rectypespresent))

['hdata-2021-03-28_10-48-33.log']
Extracting knowndate from filename: 2021-03-28
Error: Should be same midnight stamps 2021-03-28 00:00:00 2021-03-27 00:00:00
End found End(8811,3247)
Z:8142, U:29, S:345, X:295, aZ:2485, aQ:72, aF:617, aV:72
linAdifftime 48100760.63192183 count 614
Made for Q 0 last index 0
Made for V 0 last index 0
Loading aQV as no pQV records
Made for aQ 72 last index 72
pCattrname aQ
Setting aRdatetime0 2021-03-28 20:24:23.918916666 from 2021-03-27 21:24:23.963000 with std 14.95
Made for aV 72 last index 72
pCattrname aV
Made for Z 8142 last index 8142
Warning, using guessed (not GPS) timing corrected value on Z 8142
pCattrname pZ
Made for U 29 last index 29
Warning, using guessed (not GPS) timing corrected value on U 29
pCattrname pU
Made for S 345 last index 345
Warning, using guessed (not GPS) timing corrected value on S 345
pCattrname pS
Made for X 295 last index 295
Warning, using guessed (not GPS) timing corrected value on X 295
pCattrname pX
Made for aZ 2485

8

In [19]:
try:
    influxdbconfig = dict(l.split()  for l in open("../influxdb_password.txt").readlines()  if l.strip())
except FileNotFoundError:
    print("Missing password file, creating blank one to fill in")
    fpass = open("../influxdb_password.txt", "w")
    fpass.write("influxdbuser      hanglog\n")
    fpass.write("influxdbpassword      password-goes-here\n")
    fpass.close()
    influxdbconfig["influxdbpassword"] = "password-goes-here"
if influxdbconfig["influxdbpassword"] == "password-goes-here":
    print("please set the password in the file influxdb_password.txt")
    #sys.exit()



In [22]:
import influxdb
client = influxdb.InfluxDBClient(host='influxdb.doesliverpool.xyz', port=8086, 
                                 username=influxdbconfig["influxdbuser"], password=influxdbconfig["influxdbpassword"])
print("databases present", client.get_list_database())
client.switch_database('hanglogdb')

databases present [{'name': 'ming_default'}, {'name': '_internal'}, {'name': 'julianhousedb'}, {'name': 'rosebankdb'}, {'name': 'hanglogdb'}]


In [74]:
tags = { "device":"sopermodule", "uploaddate":datetime.datetime.now().isoformat()[:10] }
measurement = "hanglog"

In [85]:

for k in rectypespresent:
    fieldkeys = None
    precs = None
    if k in "UZSXYQVGFW":
        fieldkeys = hacktrack.loaders.__getattribute__("recargs"+k)[2]
        precs = fd.__getattribute__("p"+k)
        measurement = "p"+k
    elif k[0] == "a" and k[1] in "ZQFV":
        fieldkeys = hacktrack.loaders.__getattribute__("recargsA"+k[1])[2]
        precs = fd.__getattribute__(k)
        measurement = k
    else:
        print("skipping", k)
        continue
    if k == "Z" or k == "Y":
        precs = precs[~precs["bad"]]
        precs.loc[:,"s"] = precs["s"].astype(int)
        if (precs["mx"] == 0).all() and (precs["mx"] == 0).all() and (precs["mz"] == 0).all():
            fieldkeys.remove("mx"); fieldkeys.remove("my"); fieldkeys.remove("mz")
        fieldkeys.extend(["pitch", "roll", "heading"])

    #print(fieldkeys)
    #print(precs.head())

    tags["logchannel"] = k
    records = [ ]
    for idx, row in precs.iterrows():
        #print(row)
        fields = dict((f, row[f])  for f in fieldkeys)
        record = { "measurement":measurement, "tags":tags, "time":idx.isoformat(), "fields":fields }
        #print(record)
        records.append(record)
    print("writing %d records to channel %s" % (len(records), k))
    client.write_points(records)        
    print("  done")
print("all done")

writing 8122 records to channel Z
  done
writing 29 records to channel U
  done
writing 345 records to channel S
  done
writing 295 records to channel X
  done
writing 2485 records to channel aZ
  done
writing 72 records to channel aQ
  done
writing 617 records to channel aF
  done
writing 72 records to channel aV
  done
all done


In [86]:
fd.aF

,Pr
2021-03-29 09:48:33.370916666,101939.0
2021-03-29 09:48:33.566916666,101930.0
2021-03-29 09:48:33.767916666,101937.0
2021-03-29 09:48:33.974916666,101934.0
2021-03-29 09:48:34.166916666,101934.0
...,...
2021-03-29 09:50:35.776916666,101937.0
2021-03-29 09:50:36.006916666,101935.0
2021-03-29 09:50:36.167916666,101932.0
2021-03-29 09:50:36.366916666,101935.0


In [87]:
fd.pZ

,ax,ay,az,gx,gy,gz,q0,q1,q2,q3,s,mx,my,mz,iqsq,pitch,roll,heading,bad
2021-03-28 10:48:33.373632,0.00,-0.07,-0.02,-2.81,3.25,8.81,15948.0,2685.0,2534.0,685.0,49.0,0.000000,0.008889,-0.010000,3.725251e-09,19.388135,-16.701658,82.073582,False
2021-03-28 10:48:33.384632,-0.01,-0.05,-0.05,-2.81,3.25,8.81,15948.0,2685.0,2534.0,685.0,49.0,0.002222,0.002222,0.000000,3.725251e-09,19.388135,-16.701658,82.073582,False
2021-03-28 10:48:33.396632,-0.10,0.02,0.08,-2.81,3.25,8.81,15948.0,2685.0,2534.0,685.0,49.0,0.001111,-0.007778,0.007778,3.725251e-09,19.388135,-16.701658,82.073582,False
2021-03-28 10:48:33.407632,-0.06,0.00,0.04,-2.81,3.25,8.81,15948.0,2685.0,2534.0,686.0,49.0,0.000000,-0.008889,0.011111,3.725232e-09,19.389179,-16.700374,82.066500,False
2021-03-28 10:48:33.418632,0.03,-0.05,0.01,-2.81,3.25,8.81,15948.0,2685.0,2534.0,685.0,49.0,0.001111,-0.002222,0.003333,3.725251e-09,19.388135,-16.701658,82.073582,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-28 10:50:36.840632,-0.04,0.02,0.01,-2.68,3.17,8.88,15949.0,2534.0,2491.0,1199.0,48.0,0.000000,0.003333,-0.004444,3.725304e-09,18.866694,-15.864781,78.637787,False
2021-03-28 10:50:36.852632,-0.01,0.02,0.02,-2.68,3.17,8.88,15949.0,2534.0,2491.0,1199.0,48.0,-0.001111,-0.001111,-0.001111,3.725304e-09,18.866694,-15.864781,78.637787,False
2021-03-28 10:50:36.864632,-0.06,0.01,0.01,-2.68,3.17,8.88,15949.0,2534.0,2491.0,1199.0,48.0,0.000000,-0.004444,0.002222,3.725304e-09,18.866694,-15.864781,78.637787,False
2021-03-28 10:50:36.876632,-0.04,0.02,-0.03,-2.68,3.17,8.88,15949.0,2534.0,2491.0,1199.0,48.0,0.000000,0.003333,-0.003333,3.725304e-09,18.866694,-15.864781,78.637787,False
